In [3]:
from music.models import Album, Song

In [4]:
Album.objects.bulk_create([
    Album(album_title="Humbug", artist="Arctic Monkeys"),
    Album(album_title="Is This It", artist="The Strokes"),
    Album(album_title="Happier Than Ever", artist="Billie Eilish"),
    Album(album_title="Dreamland", artist="Glass Animals"),
])

[<Album: Album object (1)>,
 <Album: Album object (2)>,
 <Album: Album object (3)>,
 <Album: Album object (4)>]

In [13]:
humbug = Album.objects.get(album_title="Humbug", artist="Arctic Monkeys")
is_this_it = Album.objects.get(album_title="Is This It", artist="The Strokes")
hte = Album.objects.get(album_title="Happier Than Ever", artist="Billie Eilish")
dreamland = Album.objects.get(album_title="Dreamland", artist="Glass Animals")

In [6]:
Song.objects.create(song_title="Drag Queen")
Song.objects.create(song_title="Everything I Wanted")
Song.objects.create(song_title="I Dont Wanna Talk (I Just Wanna Dance)")

<Song: Song object (3)>

In [10]:
is_this_it.songs.create(song_title="Last Nite")

<Song: Song object (4)>

In [11]:
hte.songs.create(song_title="NDA")

<Song: Song object (5)>

In [14]:
dreamland.songs.create(song_title="Tangerine")

<Song: Song object (6)>

In [12]:
print(Song.objects.select_related("album").query)

SELECT "restaurants_song"."id", "restaurants_song"."song_title", "restaurants_song"."album_id", "restaurants_album"."id", "restaurants_album"."album_title", "restaurants_album"."artist" FROM "restaurants_song" LEFT OUTER JOIN "restaurants_album" ON ("restaurants_song"."album_id" = "restaurants_album"."id")


In [16]:
print(Song.objects.select_related("album").filter(album__isnull=False).query)

SELECT "restaurants_song"."id", "restaurants_song"."song_title", "restaurants_song"."album_id", "restaurants_album"."id", "restaurants_album"."album_title", "restaurants_album"."artist" FROM "restaurants_song" INNER JOIN "restaurants_album" ON ("restaurants_song"."album_id" = "restaurants_album"."id") WHERE "restaurants_song"."album_id" IS NOT NULL


In [22]:
print(Song.objects.filter(album__album_title="Dreamland").query)

SELECT "restaurants_song"."id", "restaurants_song"."song_title", "restaurants_song"."album_id" FROM "restaurants_song" INNER JOIN "restaurants_album" ON ("restaurants_song"."album_id" = "restaurants_album"."id") WHERE "restaurants_album"."album_title" = Dreamland


In [5]:
print(Song.objects.exclude(album__album_title="Dreamland").query)

SELECT "restaurants_song"."id", "restaurants_song"."song_title", "restaurants_song"."album_id" FROM "restaurants_song" LEFT OUTER JOIN "restaurants_album" ON ("restaurants_song"."album_id" = "restaurants_album"."id") WHERE NOT ("restaurants_album"."album_title" = Dreamland AND "restaurants_album"."album_title" IS NOT NULL AND "restaurants_song"."album_id" IS NULL)


In [8]:
print(Song.objects.exclude(album__isnull=False, album__album_title="Dreamland").query)

SELECT "restaurants_song"."id", "restaurants_song"."song_title", "restaurants_song"."album_id" FROM "restaurants_song" LEFT OUTER JOIN "restaurants_album" ON ("restaurants_song"."album_id" = "restaurants_album"."id") WHERE NOT ("restaurants_album"."album_title" = Dreamland AND "restaurants_album"."album_title" IS NOT NULL AND "restaurants_song"."album_id" IS NOT NULL)


In [11]:
print(Album.objects.filter(songs__song_title="NDA").query)

SELECT "restaurants_album"."id", "restaurants_album"."album_title", "restaurants_album"."artist" FROM "restaurants_album" INNER JOIN "restaurants_song" ON ("restaurants_album"."id" = "restaurants_song"."album_id") WHERE "restaurants_song"."song_title" = NDA


In [15]:
Album.objects.all()

<QuerySet [<Album: Album object (1)>, <Album: Album object (2)>, <Album: Album object (3)>, <Album: Album object (4)>]>